# Limpieza de Jugadores (Player_Cleaning)
Este script procesa `player_enriched_final.csv` aplicando las recomendaciones de limpieza:
- Eliminar columnas internas irrelevantes
- Unificar duplicadas priorizando menor cantidad de nulos
- Rellenar texto con 'Desconocido'
- Filtrar jugadores con más del 60% de valores nulos


In [3]:
# ==========================================================
# SCRIPT DE LIMPIEZA DE JUGADORES (PLAYER_ENRICHED_FINAL)
# ==========================================================

import os
import pandas as pd
import numpy as np

# ----------------------------------------------------------
# PASO 1: CARGA DE DATOS
# ----------------------------------------------------------
path_base = r"C:\\Users\\juanl\\Downloads\\final\\csv"
input_file = os.path.join(path_base, "player_enriched_final.csv")
output_file = os.path.join(path_base, "player_enriched_final2.csv")

print(f"📥 Cargando dataset: {input_file}")
df = pd.read_csv(input_file, low_memory=False)
print(f"✅ Cargado correctamente ({df.shape[0]} filas, {df.shape[1]} columnas)")

# ----------------------------------------------------------
# PASO 2: ELIMINAR COLUMNAS INTERNAS IRRELEVANTES
# ----------------------------------------------------------
cols_irrelevantes = [c for c in df.columns if any(x in c.lower() for x in ["slug", "display", "code", "url", "link"])]
if cols_irrelevantes:
    df.drop(columns=cols_irrelevantes, inplace=True)
    print(f"🗑️ Eliminadas columnas irrelevantes: {cols_irrelevantes}")

# ----------------------------------------------------------
# PASO 3: UNIFICAR COLUMNAS DUPLICADAS (MENOS NULOS)
# ----------------------------------------------------------
def unificar_columna(df, base_name):
    col_matches = [c for c in df.columns if base_name in c.lower()]
    if len(col_matches) > 1:
        col_mejor = min(col_matches, key=lambda c: df[c].isna().mean())
        for c in col_matches:
            if c != col_mejor:
                df[col_mejor] = df[col_mejor].combine_first(df[c])
                df.drop(columns=[c], inplace=True)
        print(f"🔄 Unificadas columnas {col_matches} → {col_mejor}")
    return df

for base in ["height", "weight", "team_id", "birth_state", "birth_country"]:
    df = unificar_columna(df, base)

# ----------------------------------------------------------
# PASO 3.1: ELIMINAR DIRECTAMENTE 'id' (CONSERVAR 'person_id')
# ----------------------------------------------------------
if 'id' in df.columns:
    df.drop(columns=['id'], inplace=True)
    print("🧹 Eliminada columna 'id' (se conserva 'person_id' como identificador único).")

# ----------------------------------------------------------
# PASO 4: ELIMINAR JUGADORES CON MÁS DE 5 COLUMNAS VACÍAS
# ----------------------------------------------------------
antes = len(df)
df = df[df.isna().sum(axis=1) <= 5]
despues = len(df)
print(f"📉 Jugadores eliminados por exceso de vacíos: {antes - despues} ({round((antes - despues)/antes*100, 2)}%)")

# ----------------------------------------------------------
# PASO 5: CONVERTIR COLUMNAS NUMÉRICAS A INT32 / INT64
# ----------------------------------------------------------
num_cols = df.select_dtypes(include=np.number).columns

for col in num_cols:
    if (df[col].dropna() % 1 == 0).all():
        try:
            df[col] = df[col].astype("Int32")
        except Exception:
            df[col] = df[col].astype("Int64", errors="ignore")
    else:
        df[col] = df[col].astype("float64", errors="ignore")

print(f"🔢 Columnas numéricas convertidas ({len(num_cols)} columnas).")

# ----------------------------------------------------------
# PASO 6: RELLENAR TEXTO CON 'DESCONOCIDO'
# ----------------------------------------------------------
cat_cols = df.select_dtypes(include="object").columns
for col in cat_cols:
    df[col] = df[col].fillna("Desconocido")

print(f"🧩 Rellenados valores nulos en columnas de texto ({len(cat_cols)} columnas).")

# ----------------------------------------------------------
# PASO 7: GUARDAR RESULTADO FINAL
# ----------------------------------------------------------
df.to_csv(output_file, index=False)
print(f"💾 Guardado → {output_file}")
print(f"✅ Limpieza finalizada ({df.shape[0]} filas, {df.shape[1]} columnas)")


📥 Cargando dataset: C:\\Users\\juanl\\Downloads\\final\\csv\player_enriched_final.csv
✅ Cargado correctamente (4899 filas, 35 columnas)
🗑️ Eliminadas columnas irrelevantes: ['display_first_last', 'display_last_comma_first', 'display_fi_last', 'player_slug', 'team_code', 'playercode']
🧹 Eliminada columna 'id' (se conserva 'person_id' como identificador único).
📉 Jugadores eliminados por exceso de vacíos: 663 (13.53%)
🔢 Columnas numéricas convertidas (8 columnas).
🧩 Rellenados valores nulos en columnas de texto (20 columnas).
💾 Guardado → C:\\Users\\juanl\\Downloads\\final\\csv\player_enriched_final2.csv
✅ Limpieza finalizada (4236 filas, 28 columnas)
